# CONUS404 Explorer Panel App
Using hvplot and Panel from the [holoviz](https://holoviz.org) suite

In [2]:
import xarray as xr
import cf_xarray 
import metpy
import intake
import numpy as np
import hvplot.xarray
from geoviews import tile_sources as gvts
import panel as pn

pn.extension()

# panel config
pn.config.throttled = True # will keep sliders from triggering until mouse released

In [3]:
url = 'https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml'

In [4]:
hytest_cat = intake.open_catalog(url)
list(hytest_cat)

['conus404-catalog',
 'conus404-drb-eval-tutorial-catalog',
 'nhm-v1.0-daymet-catalog',
 'nhm-v1.1-c404-bc-catalog',
 'nhm-v1.1-gridmet-catalog',
 'trends-and-drivers-catalog',
 'nhm-prms-v1.1-gridmet-format-testing-catalog',
 'nwis-streamflow-usgs-gages-onprem',
 'nwis-streamflow-usgs-gages-osn',
 'nwm21-streamflow-usgs-gages-onprem',
 'nwm21-streamflow-usgs-gages-osn',
 'nwm21-streamflow-cloud',
 'geofabric_v1_1-zip-osn',
 'geofabric_v1_1_POIs_v1_1-osn',
 'geofabric_v1_1_TBtoGFv1_POIs-osn',
 'geofabric_v1_1_nhru_v1_1-osn',
 'geofabric_v1_1_nhru_v1_1_simp-osn',
 'geofabric_v1_1_nsegment_v1_1-osn',
 'gages2_nndar-osn',
 'wbd-zip-osn',
 'huc12-geoparquet-osn',
 'huc12-gpkg-osn',
 'nwm21-scores',
 'lcmap-cloud',
 'rechunking-tutorial-osn',
 'pointsample-tutorial-sites-osn',
 'pointsample-tutorial-output-osn']

In [5]:
# open the conus404 sub-catalog
cat = hytest_cat['conus404-catalog']
list(cat)

['conus404-hourly-onprem',
 'conus404-hourly-cloud',
 'conus404-hourly-osn',
 'conus404-daily-diagnostic-onprem',
 'conus404-daily-diagnostic-cloud',
 'conus404-daily-diagnostic-osn',
 'conus404-daily-onprem',
 'conus404-daily-cloud',
 'conus404-daily-osn',
 'conus404-monthly-onprem',
 'conus404-monthly-cloud',
 'conus404-monthly-osn',
 'conus404-hourly-ba-osn',
 'conus404-daily-ba-osn']

In [6]:
dataset = "conus404-hourly-cloud"
cat[dataset].storage_options.update({"profile": "developer"})
# dataset = "conus404-hourly-osn"

This app views the entire spatial domain, so tell Dasks to read in bigger chunks to speed up loading. The native chunk size in the Zarr dataset is `{'x':175, 'y':175} ` so pick multiples of that.  This reduces the time to load the entire domain on a 2 core machine on AWS us-west-2 from 15s to 5s. 

In [7]:
cat[dataset].kwargs = ({'chunks':{'x':2*175, 'y':2*175}})
cat[dataset].kwargs

{'chunks': {'x': 350, 'y': 350}}

In [8]:
ds = cat[dataset].to_dask()

/home/alaws/mambaforge/envs/panel/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


Add `standard_name='time'` so `cf_xarray` can find the time coordinate

In [9]:
ds.time.attrs['standard_name'] = 'time'
# ds.x.attrs['standard_name'] = 'longitude'
# ds.y.attrs['standard_name'] = 'latitude'

Find data variables that have a time dimension (but are not time `bounds` variables)

In [10]:
time_vars = []
for var in ds.data_vars:
    if len(ds[var].dims) > 0:
        if 'time' in ds[var].dims[0] and not 'bounds' in var:
            time_vars.append(var)

In [11]:
print(len(time_vars))

89


In [12]:
init_var = 'T2'

In [13]:
ds[init_var]

<xarray.DataArray 'T2' (time: 368064, y: 1015, x: 1367)>
dask.array<open_dataset-T2, shape=(368064, 1015, 1367), dtype=float32, chunksize=(144, 350, 350), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1979-10-01 ... 2021-09-25T23:00:00
  * x        (x) float64 -2.732e+06 -2.728e+06 ... 2.728e+06 2.732e+06
  * y        (y) float64 -2.028e+06 -2.024e+06 -2.02e+06 ... 2.024e+06 2.028e+06
Attributes:
    description:         TEMP at 2 M
    integration_length:  K
    long_name:           Temperature at 2 meters
    units:               K

Use Metpy to parse the CRS

In [14]:
ds = ds.metpy.parse_cf()

In [15]:
ds

<xarray.Dataset>
Dimensions:         (time: 368064, x: 1367, y: 1015, bottom_top_stag: 51,
                     bottom_top: 50, soil_layers_stag: 4, x_stag: 1368,
                     y_stag: 1016, snow_layers_stag: 3, snso_layers_stag: 7)
Coordinates:
  * time            (time) datetime64[ns] 1979-10-01 ... 2021-09-25T23:00:00
  * x               (x) float64 -2.732e+06 -2.728e+06 ... 2.728e+06 2.732e+06
  * y               (y) float64 -2.028e+06 -2.024e+06 ... 2.024e+06 2.028e+06
    metpy_crs       object Projection: lambert_conformal_conic
Dimensions without coordinates: bottom_top_stag, bottom_top, soil_layers_stag,
                                x_stag, y_stag, snow_layers_stag,
                                snso_layers_stag
Data variables: (12/163)
    ACDEWC          (time, y, x) float32 dask.array<chunksize=(144, 350, 350), meta=np.ndarray>
    ACDRIPR         (time, y, x) float32 dask.array<chunksize=(144, 350, 350), meta=np.ndarray>
    ACDRIPS         (time, y, x) float32 dask.array<chunksize=(144, 350, 350), meta=np.ndarray>
    ACECAN          (time, y, x) float32 dask.array<chunksize=(144, 350, 350), meta=np.ndarray>
    ACEDIR          (time, y, x) float32 dask.array<chunksize=(144, 350, 350), meta=np.ndarray>
    ACETLSM         (time, y, x) float32 dask.array<chunksize=(144, 350, 350), meta=np.ndarray>
    ...              ...
    lat             (y, x) float32 dask.array<chunksize=(350, 350), meta=np.ndarray>
    lat_u           (y, x_stag) float32 dask.array<chunksize=(350, 175), meta=np.ndarray>
    lat_v           (y_stag, x) float32 dask.array<chunksize=(175, 350), meta=np.ndarray>
    lon             (y, x) float32 dask.array<chunksize=(350, 350), meta=np.ndarray>
    lon_u           (y, x_stag) float32 dask.array<chunksize=(350, 175), meta=np.ndarray>
    lon_v           (y_stag, x) float32 dask.array<chunksize=(175, 350), meta=np.ndarray>
Attributes: (12/148)
    AER_ANGEXP_OPT:                  1
    AER_ANGEXP_VAL:                  1.2999999523162842
    AER_AOD550_OPT:                  1
    AER_AOD550_VAL:                  0.11999999731779099
    AER_ASY_OPT:                     1
    AER_ASY_VAL:                     0.8999999761581421
    ...                              ...
    WEST-EAST_PATCH_START_STAG:      1
    WEST-EAST_PATCH_START_UNSTAG:    1
    W_DAMPING:                       1
    YSU_TOPDOWN_PBLMIX:              0
    history:                         Tue Mar 29 16:35:22 2022: ncrcat -A -vW ...
    history_of_appended_files:       Tue Mar 29 16:35:22 2022: Appended file ...

In [16]:
crs = ds.T2.metpy.cartopy_crs

Create widget for variable selection

In [17]:
var_select = pn.widgets.Select(name='CONUS404 Variables:', options=time_vars, value=init_var)

Create widget for basemap selection

In [18]:
base_map_select = pn.widgets.Select(name='Basemap:', options=gvts.tile_sources, value=gvts.OSM)

The `plot` function below creates the `hvplot` panel layout object.   ROMS is on a C-Grid and variables have different coordinates depending on where they are on the grid.   So we use `cf-xarray` to determine the time, depth, longitude and latitude coordinates for each variable.  

We specify a basemap, specify the CRS for the selected variable, and indicate we want to `rasterize` the plot so that we can render massive meshes in the browser. 

We also specify the `groupby` parameter as the list of dimensions that remains after we remove Y and X: `ds[var].dims[:-2]`, which automatically handles variables with either dimensions [T, Y, X] or [T, Z, Y, X].  

We also specify which `bokeh` controls we want to be active by default:  the `wheel_zoom` and `pan` controls.

We also change the default slider to a selection widget for the time dimension (and vertical dimension if it exists) so that specific values are easy to select.  See https://stackoverflow.com/a/54912917/2005869

In [19]:
time_vals = 48

One might think that it would be faster to make an image map using the projected coordinates instead of quadmesh with 2D lon,lat coordinates, but it seems the same.   And QuadMesh is a nicer looking map. 

In [20]:
@pn.depends(var_select, base_map_select)
def plot(var, base_map):
    extra_dims = list(ds[var].dims[:-2])
    da = ds[var].cf.isel(T=slice(-time_vals,-1)).unify_chunks()

    mesh = da.hvplot(x='x', y='y', rasterize=True, crs=crs, title=var, attr_labels=False, 
                    cmap='turbo', width=600, grid=True, xlabel='Longitude', ylabel='Latitude',
                          height=600).opts(alpha=0.7)

    return pn.panel(mesh * base_map, widgets={extra_dims[0]: pn.widgets.Select})

In [21]:
# @pn.depends(var_select, base_map_select)
# def plot(var, base_map):
#     extra_dims = list(ds[var].dims[:-2])
#     da = ds[var].cf.isel(T=slice(-time_vals,-1)).unify_chunks().load()

#     mesh = da.hvplot.quadmesh(x=da.cf['longitude'].name, y=da.cf['latitude'].name,
#                     rasterize=True, geo=True, title=var, attr_labels=False, 
#                     cmap='turbo', grid=True, xlabel='Longitude', ylabel='Latitude',
#                           frame_height=400).opts(alpha=0.7)

#     return pn.panel(mesh * base_map, widgets={extra_dims[0]: pn.widgets.Select})

In [22]:
col = pn.Column(base_map_select, var_select, plot)

We use `.servable()` below not only to display the panel object, but to make the panel servable outside the notebook via:

`panel serve CONUS404_Panel_App.ipynb`

In [23]:
col.servable('CONUS404 Dashboard')

BokehModel(combine_events=True, render_bundle={'docs_json': {'d0158a2e-a6aa-4c83-8d22-6d5dc801b0f6': {'version…